In [ ]:
!pip install pandas transformers guidance numpy evaluate rouge-score nltk py-rouge accelerate

In [1]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 46.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [2]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import json
import nltk
import numpy as np
import csv
import warnings
from typing import Any, Dict, Tuple
import re
from datasets import load_dataset
import os
from sklearn.metrics import classification_report


from transformers import AutoTokenizer, pipeline


warnings.filterwarnings("ignore", category=UserWarning, message="A new version of the following files was downloaded from")

# Dataset

In [3]:
from datasets import load_dataset

#load the test data for the cross-model evaluation of the generation step for all language pairs
data = load_dataset("Eloquent/HalluciGen-Translation", "cross_model_evaluation")


print(data.keys())





Some datasets params were ignored: ['sep']. Make sure to use only valid params for the dataset builder and to have a up-to-date version of the `datasets` library.


Generating cross_model_evaluation_de_en split:   0%|          | 0/600 [00:00<?, ? examples/s]

Generating cross_model_evaluation_en_de split:   0%|          | 0/600 [00:00<?, ? examples/s]

Generating cross_model_evaluation_fr_en split:   0%|          | 0/600 [00:00<?, ? examples/s]

Generating cross_model_evaluation_en_fr split:   0%|          | 0/600 [00:00<?, ? examples/s]

dict_keys(['cross_model_evaluation_de_en', 'cross_model_evaluation_en_de', 'cross_model_evaluation_fr_en', 'cross_model_evaluation_en_fr'])


In [4]:
data_de_en = data['cross_model_evaluation_de_en'].to_pandas()
data_en_de = data['cross_model_evaluation_en_de'].to_pandas()
data_fr_en = data['cross_model_evaluation_fr_en'].to_pandas()
data_en_fr = data['cross_model_evaluation_en_fr'].to_pandas()

display(data_de_en.head())
display(data_en_de.head())

,id,source,hyp1,hyp2
0,0,"""Perry erklärte, er werde „nach Texas zurückke...",Perry declared that he would'return to Texas t...,Perry declared that he would return to Texas t...
1,1,"Bachmann, die im August die Ames-Probeumfrage ...","Bachmann, who won the Ames straw poll in Augus...","Bachmann, who won the Ames straw poll in Augus..."
2,2,Die Kruste ist auf der nahen Seite etwa 70 km ...,The crust is on the nearby side about 70 km th...,The crust is approximately 70 km thick on the ...
3,3,Der Anwalt forderte die Wirtschaftsprüferin au...,The lawyer demanded that the auditor provide d...,The lawyer demanded the auditor to present the...
4,4,"Diese Art ist in der Schweiz, in Frankreich un...",This type of art is represented in Switzerland...,This type of art is represented in Switzerland...


,id,source,hyp1,hyp2
0,0,The pirates were attacking the ship in order t...,"Die Piraten griffen das Schiff an, um die Prin...","Die Piraten griffen das Schiff an, um die Prin..."
1,1,"Curtis Cooper, a mathematician and computer sc...","Curtis Cooper, a mathematician and computer sc...","Curtis Cooper, ein Mathematiker und Informatik..."
2,2,In the summer of 1924 he went to Smackover in ...,Im Sommer 1924 machte er Urlaub in Smackover i...,Im Sommer 1924 ging er nach Smackover in Süd-A...
3,3,I am informed that yesterday evening a frigate...,"1. Ich bin informiert, dass gestern Abend ein ...","Ich erfuhr, dass gestern Abend ein Fregattenbo..."
4,4,"Heels should be low and wide. Sand, gravel or ...","Absätze sollten niedrig und breit sein. Sand, ...",Absätze sollten sehr niedrig und sehr breit se...


In [7]:
data_de_en.tail(5)

,id,source,hyp1,hyp2
595,595,Seit seiner Gründung hat sich The Onion zu ein...,"Since its founding, The Onion has developed in...","Since its founding, The Onion has developed in..."
596,596,Das Fliegen in der Nähe eines Flughafens oder ...,"""Flying near an airport or over a crowd of peo...",It is always a good idea to fly near an airpor...
597,597,Der Ponoru ist eine Nebenfluss des Horezu in R...,The Pony is a sidekick of the Horezu in Romania.,The Ponoru is a tributary of the Horezu in Rom...
598,598,Huhne und Pryce sollen am 16. Februar vor dem ...,Huhne and Pryce are not involved in any legal ...,Huhne and Pryce are scheduled to appear before...
599,599,Die Kruste ist auf der nahen Seite etwa 70 km ...,The crust is approximately 70 km thick on the ...,The crust is on the nearby side approximately ...


## Load Model

In [ ]:
#load model Llama3
model = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import ast
def extract_json_data(result_text):
    # Find the start and end indices of the JSON data
    start_index = result_text.find('{')
    end_index = result_text.find('}') + 1  # Add 1 to include the closing brace

    # Extract the JSON data from the string
    result_text = result_text[start_index:end_index]
    #print("result text: ", result_text)

    # Replace single quotes used as delimiters with double quotes
    result_text = json.loads(result_text.replace("'", "\""))
    return result_text

In [ ]:
#save results
def save_trial_results(df, csv_filename, translation_language):
    skipped_rows = []
    # Split DataFrame into chunks of 10 rows
    chunks = np.array_split(df, len(df) // 10 if len(df) % 10 == 0 else len(df) // 10 + 1)

    for chunk in chunks:
        # Iterate through each row in the chunk
        for index, row in chunk.iterrows():
            source = row['source']
            hyp1 = row["hyp1"]
            hyp2 = row["hyp2"]

            try:
                # Generate translation
                result = detect_hallucination(source, hyp1, hyp2, translation_language)
                print("source: ", source)
                print("hyp1: ", hyp1)
                print("hyp2: ", hyp2)
                print("result: ", result)
                extracted_hallu = extract_json_data(result)
                hallu_label = extracted_hallu["label"]

                # Update DataFrame with new values
                chunk.at[index, "pred_label"] = hallu_label
            except:
                print(f"Skipping row {index} due to error in data extraction.")
                skipped_rows.append(index)
                continue

        # Append chunk to CSV file
        if not os.path.isfile(csv_filename):
            chunk.to_csv(csv_filename, mode='w', header=True, index=False)
        else:
            chunk.to_csv(csv_filename, mode='a', header=False, index=False)

        # Load CSV file
        df = pd.read_csv(csv_filename)
        #display(df)
    return skipped_rows, df

## Prompt

Given a "src" and two hypotheses "hyp1" and "hyp2", which are translated into {translation_language}. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.
      Please explain your choice.
      Provide the result in the following format: {answer_format}. Here is an example to illustrate this:

      Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.
      hyp1: In the phase, wandering ants march at night and rest during the day.
      hyp2: In the nomadic phase, wandering ants march at night and rest during the day.
      label: hyp2
      type: addition

      Detect which of the following hypotheses is the hallucination.

       Src: {source}
       hyp1 : {hyp1}
       hyp2 : {hyp2}

       Result:
  

#English-German

In [ ]:
data_en_de.head()

,id,source,hyp1,hyp2
0,0,The pirates were attacking the ship in order t...,"Die Piraten griffen das Schiff an, um die Prin...","Die Piraten griffen das Schiff an, um die Prin..."
1,1,"Curtis Cooper, a mathematician and computer sc...","Curtis Cooper, a mathematician and computer sc...","Curtis Cooper, ein Mathematiker und Informatik..."
2,2,In the summer of 1924 he went to Smackover in ...,Im Sommer 1924 machte er Urlaub in Smackover i...,Im Sommer 1924 ging er nach Smackover in Süd-A...
3,3,I am informed that yesterday evening a frigate...,"1. Ich bin informiert, dass gestern Abend ein ...","Ich erfuhr, dass gestern Abend ein Fregattenbo..."
4,4,"Heels should be low and wide. Sand, gravel or ...","Absätze sollten niedrig und breit sein. Sand, ...",Absätze sollten sehr niedrig und sehr breit se...


In [ ]:

def detect_hallucination(source, hyp1, hyp2, translation_language):
    # Generate the prompt
    answer_format = {"type": "", "label": ""}

    # prompt for translation generation task
    user_prompt = f'''Given a "src" and two hypotheses "hyp1" and "hyp2", which are translated into {translation_language}. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.
      Please explain your choice.
      Provide the result in the following format: {answer_format}. Here is an example to illustrate this:

      Src: The days in the summer can lead to problems getting sufficient sleep and associated health issues.
      hyp1: Die Tage im Sommer können zu Problemen führen, genügend Schlaf zu bekommen und damit verbundene Gesundheitsprobleme.
      hyp2: Die sehr langen Tage im Sommer können zu Problemen führen, genügend Schlaf zu bekommen und damit verbundene Gesundheitsprobleme.
      label: hyp2
      type: addition

      Detect which of the following hypotheses is the hallucination.

       Src: {source}
       hyp1 : {hyp1}
       hyp2 : {hyp2}

       Result:
       '''
    #print(user_prompt)

    messages = [
      {"role": "system", "content": "Imagine you are a classifier."},
      {"role": "user", "content": user_prompt},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=250,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    #print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
    detected_hallu = outputs[0]["generated_text"][len(prompt):]
    print("detected hallucination: ", detected_hallu)

    return detected_hallu

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/Llama3/results_llama3_en_de.csv"
skipped_rows, df = save_trial_results(data_en_de, csv_filename, "German")


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


detected hallucination:  Based on the provided source text and the two hypotheses, I detect that "hyp2" is the hallucination.

The reason for this is that "hyp2" lacks the information about the princess's treasure chest, which is present in the original source text and also in "hyp1". This suggests that "hyp2" has omitted a crucial detail, which is a type of hallucination known as "omission".

Therefore, my result is:

{'type': 'omission', 'label': 'hyp2'}

This choice is based on the analysis of the provided text and the comparison between the original source text and the two hypotheses.
source:  The pirates were attacking the ship in order to capture the princess.
hyp1:  Die Piraten griffen das Schiff an, um die Prinzessin und ihre Schatzkiste gefangen zu nehmen.
hyp2:  Die Piraten griffen das Schiff an, um die Prinzessin gefangen zu nehmen.
result:  Based on the provided source text and the two hypotheses, I detect that "hyp2" is the hallucination.

The reason for this is that "hyp2

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


detected hallucination:  {'type': 'translation', 'label': 'hyp2'}

My choice is based on the fact that the source text is in English, and "hyp1" is a correct translation of the source text. "hyp2" is a translation with some minor changes, such as using the German word "Primzahl" instead of the English word "prime number", but this is not a significant addition or modification that would be considered hallucination. Therefore, I conclude that "hyp2" is the hallucination, and the type of hallucination is "translation".
source:  Curtis Cooper, a mathematician and computer science professor at the University of Central Missouri, has discovered the largest known prime number to date on January 25.
hyp1:  Curtis Cooper, a mathematician and computer science professor at the University of Central Missouri, has discovered the largest known prime number to date on January 25th. (correct)
hyp2:  Curtis Cooper, ein Mathematiker und Informatikprofessor an der University of Central Missouri, hat am 

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

detected hallucination:  {'type': 'addition', 'label': 'hyp2'}

I chose hyp2 as the hallucination because it adds the word "wunderbare" (wonderful) which is not present in the original source text. The original text simply states that the system has learned to play all of these games, but hyp2 adds the phrase "Das wunderbare Ding an diesem ist" (The wonderful thing about this is) which is not present in the original text. This addition is an example of a hallucination, specifically an addition of information that is not present in the source text.
source:  LP: The amazing thing about this is this is, I mean, obviously, these are old games, but the system just sees what you see, the pixels, and it has the controls and it has the score, and it's learned to play all of these games, same program.
hyp1:  LP: Das ist einfach wunderbar, das ist ja, offenbar, diese Spiele sind alt, aber das System sieht genau, was du siehst, die Pixel, und es hat die Steuerung und es hat das Score, und es hat 

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


detected hallucination:  {'type': 'addition', 'label': 'hyp2'}

I chose hyp2 as the hallucination because it adds the phrase "und als wichtigen Schritt für die Gesundheitsversorgung in Tasmanien" which is not present in the original source. This phrase is not necessary to convey the same meaning as hyp1, and it seems to be an additional statement made by the author.
source:  Ever since the Federal Government stepped in to take over funding of the Mersey hospital in Devonport, Tasmania, the state government and some federal MPs have criticised this act as a stunt in the prelude to the federal election to be called by November.
hyp1:  Seitdem das Bundesministerium die Finanzierung des Mersey-Hospitals in Devonport, Tasmanien, übernommen hat, haben das Staatsministerium und einige Bundesabgeordnete dieses Vorgehen als Wahlkampfmanöver vor der von November ausgerufenen Bundestagswahl kritisiert.
hyp2:  Seitdem das Bundesministerium die Finanzierung des Mersey-Hospitals in Devonport, Tasman

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


detected hallucination:  Based on the given source and hypotheses, I detect that the hallucinated hypothesis is "hyp2".

My choice is based on the fact that the original source text does not mention anything about the height and width of heels. The source text only provides information about the conditions on roads or paths to improve traction, which is reflected in the first hypothesis ("hyp1"). The second hypothesis ("hyp2") introduces new information that is not present in the source text, specifically about the height and width of heels. This is an example of addition, as the new information is not present in the original text.

Therefore, the result is:

{'type': 'addition', 'label': 'hyp2'}
source:  Heels should be low and wide. Sand, gravel or salt (calcium chloride) is often scattered on roads or paths to improve traction.
hyp1:  Die Heels sollten niedrig und breit sein. Sand, Kiesel oder Salz (Calciumchlorid) wird oft auf den Straßen oder Wegen verstreut, um die Traktion zu ve

In [ ]:
skipped_rows

[20,
 34,
 58,
 70,
 99,
 174,
 260,
 273,
 281,
 313,
 361,
 429,
 433,
 517,
 548,
 556,
 577]

## German - English

In [ ]:

def detect_hallucination(source, hyp1, hyp2, translation_language):
    # Generate the prompt
    answer_format = {"type": "", "label": ""}

    # prompt for translation generation task
    user_prompt = f'''Given a "src" and two hypotheses "hyp1" and "hyp2", which are translated into {translation_language}. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.
      Please explain your choice.
      Provide the result in the following format: {answer_format}. Here is an example to illustrate this:

      Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.
      hyp1: In the phase, wandering ants march at night and rest during the day.
      hyp2: In the nomadic phase, wandering ants march at night and rest during the day.
      label: hyp2
      type: addition

      Detect which of the following hypotheses is the hallucination.

       Src: {source}
       hyp1 : {hyp1}
       hyp2 : {hyp2}

       Result:
       '''
    #print(user_prompt)

    messages = [
      {"role": "system", "content": "Imagine you are a classifier."},
      {"role": "user", "content": user_prompt},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=250,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    #print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
    print(outputs)
    detected_hallu = outputs[0]["generated_text"][len(prompt):]
    print("detected hallucination: ", detected_hallu)

    return detected_hallu

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/Llama3/results_llama3_de_en.csv"
skipped_rows, df = save_trial_results(data_de_en, csv_filename, "English")


Output hidden; open in https://colab.research.google.com to view.

# English-French

In [ ]:

def detect_hallucination(source, hyp1, hyp2, translation_language):
    # Generate the prompt
    answer_format = {"type": "", "label": ""}

    # prompt for translation generation task
    user_prompt = f'''Given a "src" and two hypotheses "hyp1" and "hyp2", which are translated into {translation_language}. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.
      Please explain your choice.
      Provide the result in the following format: {answer_format}. Here is an example to illustrate this:

      Src: The Madhya Police recovered the stolen laptop and mobile phone.
      hyp1: La police de Madhya Pradesh a récupéré l'ordinateur portable et le téléphone portable volés.
      hyp2: La police de Madhya a récupéré l'ordinateur portable et le téléphone portable volés.
      label: hyp1
      type: addition

      Detect which of the following hypotheses is the hallucination.

       Src: {source}
       hyp1 : {hyp1}
       hyp2 : {hyp2}

       Result:
       '''
    #print(user_prompt)

    messages = [
      {"role": "system", "content": "Imagine you are a classifier."},
      {"role": "user", "content": user_prompt},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=250,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    #print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
    print(outputs)
    detected_hallu = outputs[0]["generated_text"][len(prompt):]
    print("detected hallucination: ", detected_hallu)

    return detected_hallu

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/Llama3/results_llama3_en_fr.csv"
skipped_rows, df = save_trial_results(data_en_fr, csv_filename, "French")


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
skipped_rows

[31,
 33,
 36,
 90,
 102,
 106,
 139,
 146,
 175,
 227,
 235,
 243,
 247,
 290,
 314,
 329,
 346,
 372,
 373,
 387,
 405,
 426,
 443,
 483,
 485,
 493,
 534,
 548,
 599]

## French-English

In [ ]:

def detect_hallucination(source, hyp1, hyp2, translation_language):
    # Generate the prompt
    answer_format = {"type": "", "label": ""}

    # prompt for translation generation task
    user_prompt = f'''Given a "src" and two hypotheses "hyp1" and "hyp2", which are translated into {translation_language}. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.
      Please explain your choice.
      Provide the result in the following format: {answer_format}. Here is an example to illustrate this:

      Src: Mélangez les poudres sèches ensemble, puis, avec les mains propres et humides, pressez-les en une boule.
      hyp1: Mix the two dried powders together, then, with clean and wet hands, press them into a ball.
      hyp2: Mix the dried powders together, then, with clean and wet hands, press them into a ball
      label: hyp1
      type: addition

      Detect which of the following hypotheses is the hallucination.

       Src: {source}
       hyp1 : {hyp1}
       hyp2 : {hyp2}

       Result:
       '''
    #print(user_prompt)

    messages = [
      {"role": "system", "content": "Imagine you are a classifier."},
      {"role": "user", "content": user_prompt},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=250,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    #print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
    print(outputs)
    detected_hallu = outputs[0]["generated_text"][len(prompt):]
    print("detected hallucination: ", detected_hallu)

    return detected_hallu

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/Llama3/results_llama3_fr_en.csv"
skipped_rows, df = save_trial_results(data_fr_en, csv_filename, "English")


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
skipped_rows

[4, 69, 70, 76, 271, 297, 313, 400, 450, 546, 583, 591]

## Convert to comma-separated csv

In [ ]:
import pandas as pd
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/Llama3/results_llama3_de_en_final.csv"


df = pd.read_csv(csv_filename, sep=";")



In [ ]:
df.head()

,id,label
0,0,hyp2
1,1,NaN
2,2,hyp1
3,3,hyp2
4,4,hyp2


In [ ]:
len(df)

600

In [ ]:
df.to_csv("/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/Llama3/results_llama3_de_en_final.csv", mode='w', header=True, index=False)

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/Llama3/results_llama3_en_fr_final.csv"


df = pd.read_csv(csv_filename, sep=";")
df.head()


,id,label
0,0,hyp2
1,1,hyp2
2,2,hyp2
3,3,hyp2
4,4,hyp2


In [ ]:
len(df)

600

In [ ]:
df.to_csv(csv_filename, mode='w', header=True, index=False)

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/Llama3/results_llama3_fr_en_final.csv"


df = pd.read_csv(csv_filename, sep=";")
df.head()


,id,label
0,0,hyp2
1,1,hyp2
2,2,hyp1
3,3,hyp2
4,4,hyp1


In [ ]:
len(df)

600

In [ ]:
df.to_csv(csv_filename, mode='w', header=True, index=False)

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/Llama3/results_llama3_en_de_final.csv"


df = pd.read_csv(csv_filename, sep= ";")
df.head()


,id,label
0,0,hyp2
1,1,hyp2
2,2,hyp1
3,3,hyp2
4,4,hyp2


In [ ]:
df.to_csv(csv_filename, mode='w', header=True, index=False)